In [17]:
import sys
import os

# Add the root directory to the Python environment
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_dir)

In [18]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [84]:
df = pd.read_csv("../data/aggregated/aggregated_1752501542.csv.gz", compression="gzip")
df.head()

,coin,err,algo_time,algo,block,prep_time,n_nodes,n_edges,top_coin_count,avg_time,is_timeout,is_failed
0,0x89a8c847f41c0dfa6c8b88638bacca8a0b777da7,timeout: Algorithm execution timed out,12.000070,mmbf,22830752,4.768372e-07,1000,4416,1000,12.00007,True,False
1,0xfa14fa6958401314851a17d6c5360ca29f74b57b,timeout: Algorithm execution timed out,12.000070,mmbf,22830752,4.768372e-07,1000,4416,1000,NaN,True,False
2,0xee2a03aa6dacf51c18679c516ad5283d8e7c2637,timeout: Algorithm execution timed out,12.000067,mmbf,22830752,4.768372e-07,1000,4416,1000,NaN,True,False
3,0xcc4304a31d09258b0029ea7fe63d032f52e44efe,timeout: Algorithm execution timed out,12.000068,mmbf,22830752,4.768372e-07,1000,4416,1000,NaN,True,False
4,0x7afebbb46fdb47ed17b22ed075cde2447694fb9e,timeout: Algorithm execution timed out,12.000070,mmbf,22830752,4.768372e-07,1000,4416,1000,NaN,True,False


In [85]:
# Filter the dataframe to only include blocks in the range 22820000 to 22821000 (inclusive)
df = df[(df['block'] >= 22820000) & (df['block'] <= 22839999)]


In [64]:
num_blocks = df['block'].nunique()
print(num_blocks)


20000


In [92]:
# First, filter rows where 'err' contains 'negative'
neg_cycle_df = df[df['err'].str.contains('negative', case=False, na=False)]
neg_cycle_df.head()


# For each algorithm, get the set of blocks with a negative cycle
algo_block_neg = neg_cycle_df.groupby(['algo', 'top_coin_count']).count()
algo_block_neg.head()

coin      err  algo_time    block  prep_time  n_nodes  \
algo top_coin_count                                                             
bf   100             1940000  1940000    1940000  1940000    1940000  1940000   
     1000            1972006  1972006    1972006  1972006    1972006  1972006   
     10000             13107    13107      13107    13107      13107    13107   
     100000             3627     3627       3627     3627       3627     3627   

                     n_edges  avg_time  is_timeout  is_failed  
algo top_coin_count                                            
bf   100             1940000     19363     1940000    1940000  
     1000            1972006     19726     1972006    1972006  
     10000             13107       117       13107      13107  
     100000             3627        26        3627       3627

In [93]:

# For each algorithm, get the set of blocks with a negative cycle
algo_block_neg = neg_cycle_df.groupby(['algo', 'top_coin_count'])['coin'].count()

print(algo_block_neg)



# # Calculate the percentage
neg_cycle_pct = ((algo_block_neg / 2000000) * 100)
print(neg_cycle_pct)

# # Display the result as a DataFrame
# neg_cycle_pct_df = neg_cycle_pct.reset_index().rename(columns={'block': 'neg_cycle_block_pct', 0: 'neg_cycle_block_pct'})
# neg_cycle_pct_df.columns = ['algo', 'neg_cycle_block_pct']
# print("Percentage of blocks with negative cycle per algorithm:")
# neg_cycle_pct_df.head()



algo  top_coin_count
bf    100               1940000
      1000              1972006
      10000               13107
      100000               3627
Name: coin, dtype: int64
algo  top_coin_count
bf    100               97.00000
      1000              98.60030
      10000              0.65535
      100000             0.18135
Name: coin, dtype: float64


In [21]:
# Group by 'algo' and 'top_coin_count', then count timeouts, failures, total rows, and number of unique blocks
result = df.groupby(['algo', 'top_coin_count']).agg(
    timeout_count=('is_timeout', 'sum'),
    failed_count=('is_failed', 'sum'),
    total_count=('is_timeout', 'count'),
    n_blocks=('block', 'nunique')
).reset_index()

# Calculate success_count by subtracting failed and timeout from total_count
result['success_count'] = result['total_count'] - result['timeout_count'] - result['failed_count']

# Calculate percentages
result['success_pct'] = result['success_count'] / result['total_count'] * 100
result['timeout_pct'] = result['timeout_count'] / result['total_count'] * 100
result['failed_pct'] = result['failed_count'] / result['total_count'] * 100

# Reorder columns as: n_blocks, total_count, success_count, success_pct, timeout_count, timeout_pct, failed_count, failed_pct
cols = ['algo', 'top_coin_count', 'n_blocks', 'total_count', 'success_count', 'success_pct',
        'timeout_count', 'timeout_pct', 'failed_count', 'failed_pct']
result = result[cols]

result.head(50)



,algo,top_coin_count,n_blocks,total_count,success_count,success_pct,timeout_count,timeout_pct,failed_count,failed_pct
0,bf,100,20000,2000000,60000,3.0000,0,0.00000,1940000,97.00000
1,bf,1000,20000,2000000,27994,1.3997,0,0.00000,1972006,98.60030
2,bf,10000,20000,2000000,10634,0.5317,1976259,98.81295,13107,0.65535
3,bf,100000,20000,2000000,12884,0.6442,1983489,99.17445,3627,0.18135
4,mmbf,100,20000,2000000,1999998,99.9999,2,0.00010,0,0.00000
5,mmbf,1000,20000,2000000,0,0.0000,2000000,100.00000,0,0.00000
6,mmbf,10000,20000,2000000,0,0.0000,2000000,100.00000,0,0.00000
7,mmbf,100000,20000,2000000,0,0.0000,2000000,100.00000,0,0.00000
8,ours,100,20000,2000000,2000000,100.0000,0,0.00000,0,0.00000
9,ours,1000,20000,2000000,2000000,100.0000,0,0.00000,0,0.00000


In [35]:
# Improved LaTeX table generation for results, with corrected LaTeX formatting

from textwrap import dedent

# Table configuration
algo_order = [("ours", "Hermes"), ("mmbf", "MMBF"), ("bf", "BF")]
row_order = [100, 1000, 10000, 100000]

# Helper to get the values for a given algo and top_coin_count
def get_row(algo, tcc):
    row = result[(result['algo'] == algo) & (result['top_coin_count'] == tcc)]
    if row.empty:
        return (0, "0.0\\%", 0, "0.0\\%", 0, "0.0\\%")
    row = row.iloc[0]
    # Use string formatting that truncates instead of rounding for percentages
    def truncate(val, decimals=2):
        factor = 10 ** decimals
        return int(val * factor) / factor

    return (
        float(row['success_count']), f"{truncate(row['success_pct'], 1):.1f}\\%",
        float(row['failed_count']), f"{truncate(row['failed_pct'], 1):.1f}\\%",
        float(row['timeout_count']), f"{truncate(row['timeout_pct'], 1):.1f}\\%"
    )

# Helper to get totals for each algo
def get_total(algo):
    rows = result[result['algo'] == algo]
    solved_n = rows['success_count'].sum()
    failed_n = rows['failed_count'].sum()
    timeout_n = rows['timeout_count'].sum()
    total_n = rows['total_count'].sum()
    solved_pct = 100 * solved_n / total_n if total_n else 0.0
    failed_pct = 100 * failed_n / total_n if total_n else 0.0
    timeout_pct = 100 * timeout_n / total_n if total_n else 0.0
    return (
        float(solved_n), f"{solved_pct:.1f}\\%",
        float(failed_n), f"{failed_pct:.1f}\\%",
        float(timeout_n), f"{timeout_pct:.1f}\\%"
    )

# Build LaTeX table header with correct column formatting and line breaks
header = dedent(r"""
    \begin{sidewaystable}[htbp]
    \small
    \centering
    \begin{tabular}{    \begin{tabular}{|l|cc|cc|cc|cc|cc|cc|cc|cc|cc|}
    \hline
    & \multicolumn{6}{c|}{Hermes} & \multicolumn{6}{c|}{MMBF} & \multicolumn{6}{c|}{BF} \\
    \cline{2-19}
    & \multicolumn{2}{c|}{Solved} & \multicolumn{2}{c|}{Failed} & \multicolumn{2}{c|}{Timeout}
    & \multicolumn{2}{c|}{Solved} & \multicolumn{2}{c|}{Failed} & \multicolumn{2}{c|}{Timeout}
    & \multicolumn{2}{c|}{Solved} & \multicolumn{2}{c|}{Failed} & \multicolumn{2}{c|}{Timeout} \\
    \cline{2-19}
    \begin{tabular}[c]{@{}c@{}}Number of\\Tokens\end{tabular} & \# & \% & \# & \% & \# & \%
    & \# & \% & \# & \% & \# & \%
    & \# & \% & \# & \% & \# & \% \\
    \hline
""").strip().splitlines()

latex_table = [line.rstrip() for line in header]

# Function to format a table row
def format_latex_row(label, cells):
    # cells: list of 18 values (6 per algo: solved_n, solved_pct, failed_n, failed_pct, timeout_n, timeout_pct)
    row = [str(label)]
    for i in range(0, len(cells), 6):
        row.extend([
            f"{cells[i]:,}", f"{cells[i+1]}",
            f"{cells[i+2]:,}", f"{cells[i+3]}",
            f"{cells[i+4]:,}", f"{cells[i+5]}"
        ])
    return " & ".join(row) + r" \\"

# Add data rows
for tcc in row_order:
    row_cells = []
    for algo, _ in algo_order:
        row_cells.extend(get_row(algo, tcc))
    latex_table.append(format_latex_row(f"{tcc:,}", row_cells))

# Add total row
total_cells = []
for algo, _ in algo_order:
    total_cells.extend(get_total(algo))
latex_table.append(r"\hline")
latex_table.append(format_latex_row("Total", total_cells))
latex_table.append(r"\hline")
latex_table.append(r"\end{tabular}")
latex_table.append(r"\caption{Success, failure, and timeout counts and percentages for each algorithm and top coin count.}")
latex_table.append(r"\label{tab:success_rate_table}")
latex_table.append(r"\end{sidewaystable}")

# Print the full LaTeX table
print("\n".join(latex_table))


\begin{sidewaystable}[htbp]
\small
\centering
\begin{tabular}{    \begin{tabular}{|l|cc|cc|cc|cc|cc|cc|cc|cc|cc|}
\hline
& \multicolumn{6}{c|}{Hermes} & \multicolumn{6}{c|}{MMBF} & \multicolumn{6}{c|}{BF} \\
\cline{2-19}
& \multicolumn{2}{c|}{Solved} & \multicolumn{2}{c|}{Failed} & \multicolumn{2}{c|}{Timeout}
& \multicolumn{2}{c|}{Solved} & \multicolumn{2}{c|}{Failed} & \multicolumn{2}{c|}{Timeout}
& \multicolumn{2}{c|}{Solved} & \multicolumn{2}{c|}{Failed} & \multicolumn{2}{c|}{Timeout} \\
\cline{2-19}
\begin{tabular}[c]{@{}c@{}}Number of\\Tokens\end{tabular} & \# & \% & \# & \% & \# & \%
& \# & \% & \# & \% & \# & \%
& \# & \% & \# & \% & \# & \% \\
\hline
100 & 2,000,000.0 & 100.0\% & 0.0 & 0.0\% & 0.0 & 0.0\% & 1,999,998.0 & 99.9\% & 0.0 & 0.0\% & 2.0 & 0.0\% & 60,000.0 & 3.0\% & 1,940,000.0 & 97.0\% & 0.0 & 0.0\% \\
1,000 & 2,000,000.0 & 100.0\% & 0.0 & 0.0\% & 0.0 & 0.0\% & 0.0 & 0.0\% & 0.0 & 0.0\% & 2,000,000.0 & 100.0\% & 27,994.0 & 1.3\% & 1,972,006.0 & 98.6\% & 0.0 & 0.0\% 